In [5]:
import pandas as pd
import numpy as np
import datetime as dt

import warnings
warnings.simplefilter('ignore')

In [7]:
## Concat all monthly values for 2019 forex data, use for loop to elinmate typing.
xx = pd.read_excel(f'Raw/Excel/DAT_XLSX_EURUSD_M1_201901.xlsx', header = None)

for a in range(9):
    num = f'0{a+2}'
    if (a+2) >= 10:
        num = f'{a+2}'
    yy = pd.read_excel(f'Raw/Excel/DAT_XLSX_EURUSD_M1_2019{num}.xlsx',  header = None)
    xx = pd.concat([xx, yy])

zz = xx.reset_index(drop = True)
print('Done')

Done


In [8]:
#Clean the dataframe some
zz = zz.rename(columns={0: 'Time', 1: 'Bar OPEN Bid Quote', 2: 'Bar HIGH Bid Quote', \
                        3: 'Bar LOW Bid Quote', 4: 'Bar CLOSE Bid Quote', 5: 'Bar CLOSE Bid Quote'})
zz.head()

,Time,Bar OPEN Bid Quote,Bar HIGH Bid Quote,Bar LOW Bid Quote,Bar CLOSE Bid Quote,Bar CLOSE Bid Quote
0,2019-01-01 17:02:00,1.14598,1.14599,1.14598,1.14598,0
1,2019-01-01 17:03:00,1.14598,1.14607,1.14598,1.14607,0
2,2019-01-01 17:04:00,1.14607,1.14607,1.14606,1.14606,0
3,2019-01-01 17:05:00,1.14606,1.14621,1.14606,1.14621,0
4,2019-01-01 17:06:00,1.14619,1.14666,1.14604,1.14665,0


In [9]:
#Save raw concat file
zz.to_csv('Raw/EUR-USD-2019-RAW-CONCAT.csv', index = False)

In [10]:
#Check if nan values present. No NAN Values
zz.isna().any()

Time                   False
Bar OPEN Bid Quote     False
Bar HIGH Bid Quote     False
Bar LOW Bid Quote      False
Bar CLOSE Bid Quote    False
Bar CLOSE Bid Quote    False
dtype: bool

In [11]:
#Make lookup table to convert days to integers
day_df = pd.DataFrame({})

day_df['dayname'] = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', ' Saturday', 'Sunday']
day_df['weekday'] = [1, 2, 3, 4, 5, 6, 7]

day_df

,dayname,weekday
0,Monday,1
1,Tuesday,2
2,Wednesday,3
3,Thursday,4
4,Friday,5
5,Saturday,6
6,Sunday,7


In [22]:
#Splice time value into seperate values
t_df = zz[['Time']]
t_df['date'] = t_df['Time'].dt.strftime('%m/%d/%Y')
t_df['year'] = t_df['Time'].dt.strftime('%Y')
t_df['month'] = t_df['Time'].dt.strftime('%m')
t_df['day'] = t_df['Time'].dt.strftime('%d')
t_df['hour'] = t_df['Time'].dt.strftime('%H')
t_df['minute'] = t_df['Time'].dt.strftime('%M')
t_df['dayname'] = t_df['Time'].dt.strftime('%A')

t_df = pd.merge(t_df, day_df, how = 'left')
t_df = t_df.drop('dayname', axis = 1)
t_df.head()

,Time,date,year,month,day,hour,minute,weekday
0,2019-01-01 17:02:00,01/01/2019,2019,01,01,17,02,2
1,2019-01-01 17:03:00,01/01/2019,2019,01,01,17,03,2
2,2019-01-01 17:04:00,01/01/2019,2019,01,01,17,04,2
3,2019-01-01 17:05:00,01/01/2019,2019,01,01,17,05,2
4,2019-01-01 17:06:00,01/01/2019,2019,01,01,17,06,2


In [23]:
# Get minute-by-minute change of forex data
delta_df = zz[['Bar OPEN Bid Quote']]
delta_df['change'] = delta_df['Bar OPEN Bid Quote'].diff()
delta_df.head()

,Bar OPEN Bid Quote,change
0,1.14598,NaN
1,1.14598,0.00000
2,1.14607,0.00009
3,1.14606,-0.00001
4,1.14619,0.00013


In [24]:
#Verfiy indices match
print(t_df.index.size, delta_df.index.size)

298573 298573


In [25]:
#merge the time and delta dataframes on the index
final_df = t_df.merge(delta_df, right_index = True, left_index = True)
final_df = final_df.rename(columns = {'Time': 'time', 'Bar OPEN Bid Quote': 'actual'})
final_df.head()

,time,date,year,month,day,hour,minute,weekday,actual,change
0,2019-01-01 17:02:00,01/01/2019,2019,01,01,17,02,2,1.14598,NaN
1,2019-01-01 17:03:00,01/01/2019,2019,01,01,17,03,2,1.14598,0.00000
2,2019-01-01 17:04:00,01/01/2019,2019,01,01,17,04,2,1.14607,0.00009
3,2019-01-01 17:05:00,01/01/2019,2019,01,01,17,05,2,1.14606,-0.00001
4,2019-01-01 17:06:00,01/01/2019,2019,01,01,17,06,2,1.14619,0.00013


In [26]:
#drop first value because difference is NaN
final_df = final_df.dropna().reset_index(drop = True)
final_df.head()

,time,date,year,month,day,hour,minute,weekday,actual,change
0,2019-01-01 17:03:00,01/01/2019,2019,01,01,17,03,2,1.14598,0.00000
1,2019-01-01 17:04:00,01/01/2019,2019,01,01,17,04,2,1.14607,0.00009
2,2019-01-01 17:05:00,01/01/2019,2019,01,01,17,05,2,1.14606,-0.00001
3,2019-01-01 17:06:00,01/01/2019,2019,01,01,17,06,2,1.14619,0.00013
4,2019-01-01 17:07:00,01/01/2019,2019,01,01,17,07,2,1.14665,0.00046


In [27]:
#Verify that the first row was only one dropped, thus being only one less the previous index check
final_df.index.size

298572

In [28]:
#Save final file
final_df.to_csv('Manipulated/EURUSD-2019ForexData.csv', index = False)

FileNotFoundError: [Errno 2] No such file or directory: 'Manipulated/EURUSD-2019ForexData.csv'